# 画像ファイルを Keras / Tensorflow で読み込める形式に変換して保存する

## 概要

このノートブックでは，SageMaker 上で画像ファイルを読み込んで，学習ジョブに適した形に変換する 2 つの例を説明します．

- numpy の ndarray 形式に変換して保存
- tfrecords に変換して保存

## セットアップ

In [ ]:
# 画像を扱うモジュールである PIL をインストール
!pip install --upgrade pip && pip install pillow

In [ ]:
import glob
import os
import re
import numpy as np
from PIL import Image
import tensorflow as tf

## データ準備

png 形式の MNIST データをダウンロードする

In [ ]:
!sudo rm -r data &&\
mkdir -p data &&\
cd data &&\
git clone https://github.com/myleott/mnist_png.git &&\
tar zxf mnist_png/mnist_png.tar.gz

## numpy 形式で保存

In [ ]:
# 画像データを ndarray にして格納するための配列の定義
x_train = np.empty((0, 28, 28), int)
y_train = []

ファイルからデータを読み込んで ndarray に格納する．ここでは，データ量を間引くために，画像ファイルのうち "11" が頭につく png ファイルだけを抜き出して，訓練データとして使うことにしている．

In [ ]:
r = re.compile('.*(\d+)\/.*')
file_pathes = glob.glob('data/mnist_png/training/*/11*')

for file_path in file_pathes:
    image = Image.open(file_path)
    image_array = np.asarray(image)
    x_train = np.append(x_train, [image_array], axis=0)
    m = r.search(file_path)
    y_train.append(m.group(1))

In [ ]:
# 結果の保存
np.savez('train_data', image=x_train, label=y_train)

## tfrecords 形式で保存

Tensorflow で効率的に読みだすための tfrecords 形式に変換する場合，下記のような形でシリアライズを行う．

In [ ]:
def convert_to(images, labels, name):
    """Converts a dataset to tfrecords."""
    num_examples = len(labels)

    if images.shape[0] != num_examples:
        raise ValueError('Images size %d does not match label size %d.' %
                         (images.shape[0], num_examples))
    rows = images.shape[1]
    cols = images.shape[2]

    filename = os.path.join(name + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[rows])),
            'width': tf.train.Feature(int64_list=tf.train.Int64List(value=[cols])),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(labels[index])])),
            'image_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_raw]))}))
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
convert_to(x_train, y_train, 'train_data')